# cyBERT: a flexible log parser based on the BERT language model

## Table of Contents
* Introduction
* Generating Labeled Logs
* Subword Tokenization
* Data Loading
* Fine-tuning pretrained BERT
* Model Evaluation
* Parsing with cyBERT

## Introduction

One of the most arduous tasks of any security operation (and equally as time consuming for a data scientist) is ETL and parsing. This notebook illustrates how to train a BERT language model using a toy dataset of just 1000 previously parsed apache server logs as a labeled data. We will fine-tune a pretrained BERT model from [HuggingFace](https://github.com/huggingface) with a classification layer for Named Entity Recognition.

In [1]:
from os import path
import s3fs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.dlpack import from_dlpack
from seqeval.metrics import classification_report,accuracy_score,f1_score
from transformers import BertForTokenClassification
from tqdm import tqdm,trange
from collections import defaultdict
import pandas as pd
import numpy as np
import cupy
import cudf

## Generating Labels For Our Training Dataset

To train our model we begin with a dataframe containing parsed logs and additional `raw` column containing the whole raw log as a string. We will use the column names as our labels.

In [2]:
# download log data
APACHE_SAMPLE_CSV = "apache_sample_1k.csv"
S3_BASE_PATH = "rapidsai-data/cyber/clx"

if not path.exists(APACHE_SAMPLE_CSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + APACHE_SAMPLE_CSV, APACHE_SAMPLE_CSV)

In [3]:
logs_df = cudf.read_csv(APACHE_SAMPLE_CSV)

In [4]:
# sample parsed log
logs_df.sample(1)

error_level error_message  \
873        <NA>          <NA>   

                                                   raw   remote_host  \
873  37.59.222.30 - - [11/Feb/2018:17:44:10 +0100] ...  37.59.222.30   

    remote_logname remote_user  \
873              -           -   

                                request_header_referer  \
873  http://www.almhuette-raith.at/administrator/in...   

                             request_header_user_agent  \
873  Mozilla/5.0 (Linux; U; Android 2.2) AppleWebKi...   

    request_header_user_agent__browser__family  \
873                                    Android   

    request_header_user_agent__browser__version_string  ...  \
873                                                2.2  ...   

     request_url_username response_bytes_clf status  \
873                  <NA>               4498  200.0   

                    time_received time_received_datetimeobj  \
873  [11/Feb/2018:17:44:10 +0100]              1.518371e+12   

     time_received_isoformat time_received_tz_datetimeobj  \
873      2018-02-11T17:44:10                 1.518367e+12   

     time_received_tz_isoformat time_received_utc_datetimeobj  \
873   2018-02-11T17:44:10+01:00                  1.518367e+12   

     time_received_utc_isoformat  
873    2018-02-11T16:44:10+00:00  

[1 rows x 37 columns]

In [5]:
# sample raw log
print(logs_df.raw.loc[10])

95.108.213.19 - - [18/Jul/2018:21:53:07 +0200] "GET / HTTP/1.1" 200 10439 "-" "Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)" "-"



In [6]:
def labeler(index_no, cols):
    """
    label the words in the raw log with the column name from the parsed log
    """
    raw_split = logs_df.raw_preprocess[index_no].split()
    
    # words in raw but not in parsed logs labeled as 'other'
    label_list = ['O'] * len(raw_split) 
    
    # for each parsed column find the location of the sequence of words (sublist) in the raw log
    for col in cols:
        if str(logs_df[col][index_no]) not in {'','-','None','NaN'}:
            sublist = str(logs_df[col][index_no]).split()
            sublist_len=len(sublist)
            match_count = 0
            for ind in (i for i,el in enumerate(raw_split) if el==sublist[0]):
                # words in raw log not present in the parsed log will be labeled with 'O'
                if (match_count < 1) and (raw_split[ind:ind+sublist_len]==sublist) and (label_list[ind:ind+sublist_len] == ['O'] * sublist_len):
                    label_list[ind] = 'B-'+col
                    label_list[ind+1:ind+sublist_len] = ['I-'+col] * (sublist_len - 1)
                    match_count = 1
    return label_list

In [7]:
logs_df['raw_preprocess'] = logs_df.raw.str.replace('"','')

# column names to use as lables
cols = logs_df.columns.values.tolist()

# do not use raw columns as labels
cols.remove('raw')
cols.remove('raw_preprocess')

# using for loop for labeling funcition until string UDF capability in rapids- it is currently slow
labels = []
for indx in range(len(logs_df)):
    labels.append(labeler(indx, cols))

In [8]:
print(labels[10])

['B-remote_host', 'O', 'O', 'B-time_received', 'I-time_received', 'B-request_method', 'B-request_url', 'O', 'O', 'B-response_bytes_clf', 'O', 'B-request_header_user_agent', 'I-request_header_user_agent', 'I-request_header_user_agent', 'I-request_header_user_agent', 'O']


## Subword Labeling
We are using the `bert-base-cased` tokenizer vocabulary. This tokenizer splits our whitespace separated words further into in dictionary sub-word pieces. The model eventually uses the label from the first piece of a word as the sole label for the word, so we do not care about the model's ability to predict individual labels for the sub-word pieces. For training, the label used for these pieces is `X`. To learn more see the [BERT paper](https://arxiv.org/abs/1810.04805)

In [9]:
def subword_labeler(log_list, label_list):
    """
    label all subword pieces in tokenized log with an 'X'
    """
    subword_labels = []
    for log, tags in zip(log_list,label_list):
        temp_tags = []
        words = cudf.Series(log.split())
        words_size = len(words)
        subword_counts = words.str.subword_tokenize("resources/bert-base-cased-hash.txt", 10000, 10000,\
                                                    max_rows_tensor=words_size,\
                                                    do_lower=False, do_truncate=False)[2].reshape(words_size, 3)[:,2]
        for i, tag in enumerate(tags):
            temp_tags.append(tag)
            temp_tags.extend('X'* subword_counts[i].item())
        subword_labels.append(temp_tags)
    return subword_labels

In [10]:
subword_labels = subword_labeler(logs_df.raw_preprocess.to_arrow().to_pylist(), labels)

In [11]:
print(subword_labels[10])

['B-remote_host', 'X', 'X', 'X', 'X', 'X', 'X', 'O', 'O', 'B-time_received', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'I-time_received', 'X', 'X', 'X', 'B-request_method', 'X', 'B-request_url', 'O', 'X', 'X', 'X', 'X', 'X', 'X', 'O', 'B-response_bytes_clf', 'X', 'O', 'B-request_header_user_agent', 'X', 'X', 'X', 'X', 'X', 'I-request_header_user_agent', 'X', 'X', 'I-request_header_user_agent', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'I-request_header_user_agent', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'O']


We create a set list of all labels from our dataset, add `X` for wordpiece tokens we will not have tags for and `[PAD]` for logs shorter than the length of the model's embedding.

In [12]:
# set of labels
label_values = list(set(x for l in labels for x in l))

label_values[:0] = ['[PAD]']  

# Set a dict for mapping id to tag name
label2id = {t: i for i, t in enumerate(label_values)}
label2id.update({'X': -100})

In [13]:
print(label2id)

{'[PAD]': 0, 'I-time_received': 1, 'B-remote_host': 2, 'I-error_message': 3, 'B-request_header_referer': 4, 'B-time_received': 5, 'B-request_url': 6, 'B-response_bytes_clf': 7, 'B-request_header_user_agent': 8, 'B-error_level': 9, 'O': 10, 'B-request_method': 11, 'I-request_header_user_agent': 12, 'B-error_message': 13, 'X': -100}


In [14]:
def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l

In [15]:
padded_labels = [pad(x[:256], '[PAD]', 256) for x in subword_labels]
int_labels = [[label2id.get(l) for l in lab] for lab in padded_labels]
label_tensor = torch.tensor(int_labels).to('cuda')

# Training and Validation Datasets
For training and validation our datasets need three features. (1) `input_ids` subword tokens as integers padded to the specific length of the model (2) `attention_mask` a binary mask that allows the model to ignore padding (3) `labels` corresponding labels for tokens as integers. 

In [16]:
def bert_cased_tokenizer(strings):
    """
    converts cudf.Seires of strings to two torch tensors- token ids and attention mask with padding
    """    
    num_strings = len(strings)
    num_bytes = strings.str.byte_count().sum()
    token_ids, mask = strings.str.subword_tokenize("resources/bert-base-cased-hash.txt", 256, 256,
                                                            max_rows_tensor=num_strings,
                                                            do_lower=False, do_truncate=True)[:2]
    # convert from cupy to torch tensor using dlpack
    input_ids = from_dlpack(token_ids.reshape(num_strings,256).astype(cupy.float).toDlpack())
    attention_mask = from_dlpack(mask.reshape(num_strings,256).astype(cupy.float).toDlpack())
    return input_ids.type(torch.long), attention_mask.type(torch.long)

In [17]:
input_ids, attention_masks = bert_cased_tokenizer(logs_df.raw_preprocess)

In [18]:
# create dataset
dataset = TensorDataset(input_ids, attention_masks, label_tensor)

In [19]:
# use pytorch random_split to create training and validation data subsets
dataset_size = len(input_ids)
training_dataset, validation_dataset = random_split(dataset, (int(dataset_size*.8), int(dataset_size*.2)))

In [20]:
# create dataloader
train_dataloader = DataLoader(dataset=training_dataset, shuffle=True, batch_size=32)
val_dataloader = DataLoader(dataset=validation_dataset, shuffle=False, batch_size=1)

# Fine-tuning pretrained BERT
Download pretrained model from HuggingFace and move to GPU

In [21]:
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label2id))

# model to gpu
model.cuda()
# use multi-gpu if available
model = nn.DataParallel(model)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Define optimizer and learning rate for training

In [22]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    #fine tune all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [23]:
%%time
# using 2 epochs to avoid overfitting

epochs = 2
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)[0]
        # backward pass
        loss.sum().backward()
        # track train loss
        tr_loss += loss.sum().item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:  50%|█████     | 1/2 [00:12<00:12, 12.49s/it]

Train loss: 0.7185359191894531


Epoch: 100%|██████████| 2/2 [00:24<00:00, 12.29s/it]

Train loss: 0.02029180832207203
CPU times: user 24.4 s, sys: 108 ms, total: 24.5 s
Wall time: 24.6 s


## Model Evaluation

In [24]:
# no dropout or batch norm during eval
model.eval();

In [25]:
# Mapping id to label
id2label={label2id[key] : key for key in label2id.keys()}

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

for step, batch in enumerate(val_dataloader):
    input_ids, input_mask, label_ids = batch
        
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
        
    # Get NER predicted result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    # Get NER true result
    label_ids = label_ids.detach().cpu().numpy()
    
    # Only predict the groud truth, mask=0, will not calculate
    input_mask = input_mask.detach().cpu().numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # ground truth 
        temp_1 = []
        # Prediction
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mask=0 is PAD, do not compare
            if m: # Exclude the X label
                if id2label[label_ids[i][j]] != "X" and id2label[label_ids[i][j]] != "[PAD]": 
                    temp_1.append(id2label[label_ids[i][j]])
                    temp_2.append(id2label[logits[i][j]])
            else:
                break      
        y_true.append(temp_1)
        y_pred.append(temp_2)

print("f1 score: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
print(classification_report(y_true, y_pred,digits=3))

f1 score: 1.000000
Accuracy score: 1.000000
                           precision    recall  f1-score   support

              error_level      1.000     1.000     1.000        19
            error_message      1.000     1.000     1.000        19
              remote_host      1.000     1.000     1.000       181
   request_header_referer      1.000     1.000     1.000        96
request_header_user_agent      1.000     1.000     1.000       168
           request_method      1.000     1.000     1.000       181
              request_url      1.000     1.000     1.000       181
       response_bytes_clf      1.000     1.000     1.000       180
            time_received      1.000     1.000     1.000       200

                micro avg      1.000     1.000     1.000      1225
                macro avg      1.000     1.000     1.000      1225
             weighted avg      1.000     1.000     1.000      1225



## Saving model files for future parsing with cyBERT

In [26]:
model.module.config.id2label = id2label
model.module.config.label2id = label2id

In [27]:
 #torch.save(model.state_dict(), 'path/to/save.pth')